In [1]:
from langchain_ollama import OllamaLLM
from llama import llama_chat 

llama_model = {
    '3.2_vision11b': 'llama3.2-vision:11b',
    '3_8b': 'llama3:8b',
}

# llama local endpoint
llama_url = "http://localhost:11434/api/chat"

In [2]:
llama_chat(llama_model['3_8b'], 'Hi')

"Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?"